# Import Libraries:


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Get The Data:

In [3]:
train_ds, test_ds = tfds.load('mnist', split=['train', 'test'], as_supervised=True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


# Preprocess The Data:

In [8]:
train_data = []
train_labels = []

for image, label in train_ds:
  train_data.append(image.numpy())
  train_labels.append(label.numpy())

test_data = []
test_labels = []

for image, label in test_ds:
  test_data.append(image.numpy())
  test_labels.append(label.numpy())

In [13]:
train_data = np.asarray(train_data)
test_data = np.asarray(test_data)
train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)

In [18]:
train_data = train_data / 255
test_data = test_data / 255

# Create ImageDataGenerators:

In [19]:
train_datagen = ImageDataGenerator(rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2)

test_datagen = ImageDataGenerator(rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2)

train_datagen.fit(train_data)
test_datagen.fit(test_data)

# Create Callbacks:

In [26]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('accuracy')>0.98):
      print('98% accuracy reached, stopping training.')
      self.model.stop_training = True

callback = MyCallback()

# Create The Model:

In [27]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax'),
])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 1, 1, 64)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 64)               

In [28]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(train_datagen.flow(train_data, train_labels),
                    epochs=25,
                    validation_data=(test_datagen.flow(test_data, test_labels)),
                    callbacks=[callback])

Epoch 1/25
1875/1875 [==============================] - 99s 52ms/step - loss: 0.7069 - accuracy: 0.7609 - val_loss: 0.3722 - val_accuracy: 0.8809
Epoch 2/25
1875/1875 [==============================] - 100s 54ms/step - loss: 0.3250 - accuracy: 0.8961 - val_loss: 0.2523 - val_accuracy: 0.9178
Epoch 3/25
1875/1875 [==============================] - 100s 53ms/step - loss: 0.2589 - accuracy: 0.9157 - val_loss: 0.2081 - val_accuracy: 0.9329
Epoch 4/25
1875/1875 [==============================] - 100s 53ms/step - loss: 0.2234 - accuracy: 0.9284 - val_loss: 0.1925 - val_accuracy: 0.9367
Epoch 5/25
1875/1875 [==============================] - 101s 54ms/step - loss: 0.2051 - accuracy: 0.9348 - val_loss: 0.1999 - val_accuracy: 0.9351
Epoch 6/25
1875/1875 [==============================] - 99s 53ms/step - loss: 0.1903 - accuracy: 0.9388 - val_loss: 0.1593 - val_accuracy: 0.9476
Epoch 7/25
1875/1875 [==============================] - 102s 54ms/step - loss: 0.1813 - accuracy: 0.9420 - val_loss: 0.1